<a href="https://colab.research.google.com/github/champ-rtu/Project2/blob/main/Hate_speech_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import Libraty 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv

**There are 6 section in this project**
1. Import data
2. Preprocessing 
3. Numerical representation
4. Oversampling
5. Train the data by machine learning models
6. Evaluation in Test dataset

#**Import text data** 
Train and validataion data are imported in this section
there are two data in this project which is raw text and label data

In [2]:
#Read raw text data and label data
#Open train_text file 
x = []
x_t = []
with open('train_text.txt', 'r', encoding='utf-8') as f:
  data = f.readlines()
  
  for row in data:
    x.append(row)
    x_t.append(row)

#Open train_label file 
y = []
y_t = []
with open('train_labels.txt', 'r', encoding='utf-8') as f:
  data = f.read().split('\n')

  for row in data:
    y.append(row)
    y_t.append(row)


,Comments,Labels
0,@user nice new signage. Are you not concerned ...,0
1,A woman who you fucked multiple times saying y...,1
2,@user @user real talk do you have eyes or were...,1
3,your girlfriend lookin at me like a groupie in...,1
4,Hysterical woman like @user \n,0


In [3]:
#Read raw text data and label data
#Open val_text file 
x_val = []
with open('val_text.txt', 'r', encoding='utf-8') as f:
  data = f.readlines()
  
  for row in data:
    x.append(row)
    x_val.append(row)

#Open val_label file 
y_val = []
with open('train_labels.txt', 'r', encoding='utf-8') as f:
  data = f.read().split('\n')

  for row in data:
    y.append(row)
    y_val.append(row)

  data_val = pd.DataFrame(zip(x_val, y_val),  columns=['Comments', 'Labels'] )
data_val.head()

,Comments,Labels
0,"@user @user If book Claire wanted to ""stay in ...",0
1,After arriving in the EU refugees make protest...,1
2,😳👇 \n,1
3,@user Worst thing is if they are that stupid t...,1
4,@user Say's the HYSTERICAL woman. It is woman ...,0


In [4]:
#combine both data to dataframe
data = pd.DataFrame(zip(x, y),  columns=['Comments', 'Labels'] )
data.head()

,Comments,Labels
0,@user nice new signage. Are you not concerned ...,0
1,A woman who you fucked multiple times saying y...,1
2,@user @user real talk do you have eyes or were...,1
3,your girlfriend lookin at me like a groupie in...,1
4,Hysterical woman like @user \n,0


In [5]:
# check the number  of data
data.shape

(10000, 2)

In [6]:
#Identify label data
# It is a unbalanced class dataset
data['Labels'].value_counts()

0    5792
1    4207
        1
Name: Labels, dtype: int64

# **Preprocessing**
For preprocessing. the process consists of 
1. tokenize
2. lowercase transformation
3. Remove punctuation and stopword
4. Steming

In [7]:
#import important library for preprocessing step
import nltk
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string
import re
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
#create punctuation, stopword and stem 
punct = string.punctuation #create list of punctuation
stopwords = list(STOP_WORDS) #create list of stopword
nlp = spacy.load('en_core_web_sm')
ps = PorterStemmer() #steming
STOP_WORDS.add('user') #because the text data contain a lot of user. So "user" is added in stopword.
nlp.vocab['user'].is_stop # confirm that user is stopword 

True

In [9]:
#Create fucction for preprocessing step
#the first function is remove punctuation, covert text to lower case
def preprocessing_1(text):
  txt = re.sub(r'\d+','', text)
  txt = txt.translate(str.maketrans("","",punct))
  txt = txt.lower()
  txt = txt.strip()
  return txt

#For the second function; tokenization and remove stopwords
def preprocessing_2(text):
  txt = word_tokenize(text)
  txt = [i for i in txt if not i in stopwords]
  txt = txt = " ".join(txt)
  return txt

#The last function; stemming
def stem(text):
  txt = word_tokenize(text)
  txt = [ps.stem(i) for i in txt ]
  txt = txt = " ".join(txt)
  return txt


In [13]:
#Apply preprocessing function to the dataset
data['Comments'] = data['Comments'].apply(lambda x: preprocessing_1(x))
data['Comments'] = data['Comments'].apply(lambda x: preprocessing_2(x))
data['Comments'] = data['Comments'].apply(lambda x: stem(x))

data.head()

,Comments,Labels
0,nice new signag concern beatlemania style hyst...,0
1,woman fuck multipl time yo dick small complime...,1
2,real talk eye goug rapefug,1
3,girlfriend lookin like groupi bitch,1
4,hyster woman like,0


# **Feature Extraction section**
In the process, two feature extraction method were selected 
  1. CounterVectorizer
  2. TfidfVectorizer



In [14]:
#import importance library for feature extraction method
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


from sklearn.model_selection import train_test_split

In [17]:
#split dataset to x and y variables
y = data['Labels']
x = data['Comments']
#applied TF-IDF approach
tf=TfidfVectorizer() 
x_tf = tf.fit_transform(x)

In [18]:
#applied CountVectorizer
ct = CountVectorizer()
x_ct = ct.fit_transform(x)

In [19]:
#Split train and test dataset tf-idf approach
X_train, X_test, y_train, y_test =  train_test_split(x_tf, y, test_size = 0.3, random_state =0, shuffle = True)

In [20]:
#Split train and test dataset couter vectorizer approach
X_train_c, X_test_c, y_train_c, y_test_c =  train_test_split(x_ct, y, test_size = 0.3, random_state =0, shuffle = True)



# **Oversampling approach**


In [21]:
#import library for oversampling
from imblearn.over_sampling import RandomOverSampler

os =  RandomOverSampler( )

In [22]:
#apply Oversampling to x train with TFIDF method 
X_sm, y_sm = os.fit_sample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [23]:
#apply Oversampling to x train with CounterVectorizer method 
Xc_sm, yc_sm = os.fit_sample(X_train_c, y_train_c)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [24]:
#Check dimension after oversampling approach
X_sm.shape

(12180, 14459)

In [ ]:
#Check dimension after oversampling approach
Xc_sm.shape

(12180, 14459)

# **Train data**
Three machine learning algorithms are selected for data training 
1. Logistic regression
2. Support vector machine
3. Randomforest classifier

**Logistic regression**

In [64]:
#Import library
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [65]:
#Create logistic regression model
log_class=LogisticRegression()
# created dict which use for Gridsearch
grid={'C':10.0 **np.arange(-3,4),'penalty':['l1','l2']}

In [66]:
#Applied Gridsearch for parameter tuning and train with TF-IDF as input
log=GridSearchCV(estimator=log_class,param_grid=grid,cv=5,n_jobs=-1,scoring='f1_macro')
log.fit(X_sm,y_sm)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [67]:
#Applied Gridsearch for parameter tuning and train with Counter vector as input
log_c=GridSearchCV(estimator=log_class,param_grid=grid,cv=5,n_jobs=-1,scoring='f1_macro')
log_c.fit(Xc_sm,yc_sm)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [68]:
#Predict x_test by 2 model
y_pred_log1 = log.predict(X_test)
y_pred_log2 = log_c.predict(X_test_c)

In [ ]:
#Evaluation training set

In [70]:
#import library for evaluation
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,f1_score

In [71]:
#Evaluation the model with TF-IDF as input
f1_score(y_test, y_pred_log1, average='macro')

0.6787061011208253

In [72]:
#Evaluation the model with Count vector as input
f1_score(y_test, y_pred_log2, average='macro')

0.6928259658072133

**Random forest classifier**

In [ ]:
#import library and create Randomforest model
from sklearn.ensemble import RandomForestClassifier
clf_forest = RandomForestClassifier()

In [26]:
#create dict use for randomsearch
# Number of trees in random forest
n_estimators = [50,100,200]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

Apply random search for parameter optimization


In [30]:
#import Randomsearch library
from sklearn.model_selection import RandomizedSearchCV

In [31]:
#parameter tuning by RandomSearch
rf_random = RandomizedSearchCV(estimator = clf_forest, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)

In [32]:
#Train model with TF-IDF method
rf_random.fit(X_sm,y_sm)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 19.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 37.6min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [81]:
#Train model with Count vector method
rf_random_c = RandomizedSearchCV(estimator = clf_forest, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)
rf_random_c.fit(Xc_sm,yc_sm)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 19.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 36.7min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

**Support vector machine**

In [ ]:
#Import and create svm models for train different input (TFIDF, count vector)
from sklearn import svm
#Fit with TF-IDF
svc1 = svm.LinearSVC(C=10)
#Fit with Countervectorizer
svc2 = svm.LinearSVC(C=10)

In [ ]:
#Training data 
svc1.fit(X_sm,y_sm) #TF-IDF method
svc2.fit(Xc_sm,yc_sm) #CounterVectorizer approach

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

# **Predict and Evaluation from Test dataset**

In [33]:
#Read Test dataset with include raw text and labels
#Open test_text 
xt = []
with open('test_text.txt', 'r', encoding='utf-8') as f:
  data = f.readlines()
  
  for row in data:
    xt.append(row)

 #Open test_label
yt = []
with open('test_labels.txt', 'r', encoding='utf-8') as f:
  data = f.read().split('\n')

  for row in data:
    yt.append(row)   

#Create dataframe with combine raw text and label
data_t = pd.DataFrame(zip(xt, yt),  columns=['Comments', 'Labels'] )
data_t.head()

,Comments,Labels
0,"@user , you are correct that Reid certainly is...",0
1,Whoever just unfollowed me you a bitch \n,1
2,@user @user Those People Invaded Us!!! They DO...,1
3,"stop JUDGING bitches by there cover, jus cuz s...",1
4,how about i knock heads off and send them gift...,1


In [35]:
#Preprocessing in test file
data_t['Comments'] = data_t['Comments'].apply(lambda x: preprocessing_1(x))
data_t['Comments'] = data_t['Comments'].apply(lambda x: preprocessing_2(x))
data_t['Comments'] = data_t['Comments'].apply(lambda x: stem(x))

data_t.head()

,Comments,Labels
0,correct reid certainli weasel sadli weve got w...,0
1,unfollow bitch,1
2,peopl invad bele right americafirst open hou i...,1
3,stop judg bitch cover ju cuz bad dont mean cat...,1
4,knock head send gift wrap mom hou dumb raggedi...,1


In [37]:
#Feature extraction in test dataset by use two method
x_tf_test = tf.transform(data_t['Comments']) #TF-IDF
xc_tf_test = ct.transform(data_t['Comments']) #CounterVectorizer

**Evaluated test dataset**

support vector machine model

In [ ]:
#Evaluate support vector machine with tf-idf approach
y_svc1 = svc1.predict(x_tf_test)
print(classification_report(data_t['Labels'],y_svc1))

              precision    recall  f1-score   support

           0       0.67      0.38      0.49      1718
           1       0.47      0.75      0.58      1252

    accuracy                           0.53      2970
   macro avg       0.57      0.56      0.53      2970
weighted avg       0.59      0.53      0.52      2970



In [ ]:
#Evaluate support vector machine with countervectorize approach
y_svc2 = svc2.predict(xc_tf_test)
print(classification_report(data_t['Labels'],y_svc2))

              precision    recall  f1-score   support

           0       0.67      0.42      0.52      1718
           1       0.47      0.71      0.57      1252

    accuracy                           0.55      2970
   macro avg       0.57      0.57      0.54      2970
weighted avg       0.59      0.55      0.54      2970



logistic regression model

In [88]:
#Evaluate Logistic regression machine with tf-idf approach
y_log1 = log.predict(x_tf_test)
f1_score(data_t['Labels'], y_log1, average='macro')

0.5341770940498122

In [89]:
#Evaluate Logistic regression machine with CounterVectorize approach
y_logc = log_c.predict(xc_tf_test)
f1_score(data_t['Labels'], y_logc, average='macro')

0.5499935754816432

Randomforest model evaluation

In [93]:
#Evaluate Randomforest with tf-idf approach
y_rfc = rf_random.predict(xc_tf_test)
print(classification_report(data_t['Labels'],y_rfc))
f1_score(data_t['Labels'], y_rfc, average='macro')

              precision    recall  f1-score   support

           0       0.70      0.22      0.33      1718
           1       0.45      0.87      0.59      1252

    accuracy                           0.49      2970
   macro avg       0.58      0.55      0.46      2970
weighted avg       0.60      0.49      0.44      2970



0.463785608607547

In [38]:
#Evaluate Randomforest with CounterVectorizer approach
y_rf = rf_random.predict(x_tf_test)
print(classification_report(data_t['Labels'],y_rf))
f1_score(data_t['Labels'], y_rf, average='macro')

              precision    recall  f1-score   support

           0       0.72      0.14      0.24      1718
           1       0.44      0.92      0.60      1252

    accuracy                           0.47      2970
   macro avg       0.58      0.53      0.42      2970
weighted avg       0.60      0.47      0.39      2970



0.41711668811234154